# BERT 應用實戰

## 學習目標
- 理解 BERT 的預訓練-微調範式
- 學習使用 HuggingFace Transformers
- 實作問答系統（李宏毅 HW7）
- 掌握 BERT 微調技巧

## 對應課程
- [李宏毅 ML 2021 - BERT and its family](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW7: Question Answering (Extractive)](https://github.com/ga642381/ML2021-Spring/tree/main/HW07)

## BERT 架構概述

```
BERT 預訓練任務
├── Masked Language Model (MLM)
│   隨機遮蓋 15% 的 token，預測被遮蓋的詞
│
└── Next Sentence Prediction (NSP)
    預測兩個句子是否連續

BERT 下游任務
├── 文本分類: [CLS] token 的輸出
├── 序列標註: 每個 token 的輸出
├── 問答: 預測答案的起始和結束位置
└── 文本生成: 需要額外的解碼器
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 嘗試導入 transformers
try:
    from transformers import (
        BertTokenizer, BertModel, BertForQuestionAnswering,
        AutoTokenizer, AutoModelForQuestionAnswering,
        AdamW, get_linear_schedule_with_warmup
    )
    HAS_TRANSFORMERS = True
    print("HuggingFace Transformers 已載入")
except ImportError:
    HAS_TRANSFORMERS = False
    print("請安裝: pip install transformers")

---
## Part 1: BERT Tokenizer 基礎

BERT 使用 WordPiece tokenization，將詞分割成子詞單元。

In [ ]:
# ========== BERT Tokenizer ==========

if HAS_TRANSFORMERS:
    # 載入預訓練 tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    
    # 範例文本
    text = "深度學習是人工智慧的核心技術"
    
    # Tokenize
    tokens = tokenizer.tokenize(text)
    print(f"原文: {text}")
    print(f"Tokens: {tokens}")
    
    # 完整編碼
    encoded = tokenizer(
        text,
        padding='max_length',
        max_length=20,
        truncation=True,
        return_tensors='pt'
    )
    
    print(f"\nInput IDs: {encoded['input_ids']}")
    print(f"Attention Mask: {encoded['attention_mask']}")
    print(f"Token Type IDs: {encoded['token_type_ids']}")
    
    # 解碼
    decoded = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=True)
    print(f"\n解碼: {decoded}")
else:
    print("需要安裝 transformers 才能執行此範例")

In [ ]:
# ========== 問答任務的 Tokenization ==========

if HAS_TRANSFORMERS:
    question = "深度學習是什麼？"
    context = "深度學習是機器學習的一個分支，使用多層神經網路來學習資料的表示。"
    
    # 問答任務需要同時編碼問題和上下文
    encoded = tokenizer(
        question,
        context,
        padding='max_length',
        max_length=64,
        truncation=True,
        return_tensors='pt'
    )
    
    print("問答任務編碼:")
    print(f"  Input IDs shape: {encoded['input_ids'].shape}")
    
    # 可視化 token 結構
    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
    token_type_ids = encoded['token_type_ids'][0].tolist()
    
    print("\n結構:")
    print("[CLS] + 問題 tokens + [SEP] + 上下文 tokens + [SEP] + [PAD]...")
    print(f"\n前 20 個 tokens: {tokens[:20]}")
    print(f"Token Type IDs: {token_type_ids[:20]}")
    print("(0 = 問題, 1 = 上下文)")

---
## Part 2: BERT 模型結構

BERT 是一個多層雙向 Transformer Encoder。

In [ ]:
# ========== BERT 模型 ==========

if HAS_TRANSFORMERS:
    # 載入預訓練 BERT
    model = BertModel.from_pretrained('bert-base-chinese')
    
    # 模型結構
    print("BERT 模型結構:")
    print(f"  層數: {model.config.num_hidden_layers}")
    print(f"  隱藏維度: {model.config.hidden_size}")
    print(f"  注意力頭數: {model.config.num_attention_heads}")
    print(f"  詞彙量: {model.config.vocab_size}")
    
    # 參數量
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n總參數量: {total_params:,}")

In [ ]:
# ========== BERT 輸出 ==========

if HAS_TRANSFORMERS:
    model.eval()
    
    with torch.no_grad():
        outputs = model(**encoded)
    
    print("BERT 輸出:")
    print(f"  last_hidden_state: {outputs.last_hidden_state.shape}")
    print(f"  pooler_output: {outputs.pooler_output.shape}")
    
    # last_hidden_state: 每個 token 的表示
    # pooler_output: [CLS] token 經過 pooler 層的輸出
    
    print("\n說明:")
    print("  last_hidden_state: (batch, seq_len, hidden_size) - 所有 token 的表示")
    print("  pooler_output: (batch, hidden_size) - [CLS] 的池化表示")

---
## Part 3: Extractive QA（李宏毅 HW7）

### 任務說明

給定問題和文章，從文章中找出答案的位置（起始和結束位置）。

**輸入**：問題 + 文章

**輸出**：答案在文章中的起始和結束位置

In [ ]:
# ========== 模擬 QA 資料集 ==========

def generate_qa_data(n_samples=100):
    """
    生成模擬的 QA 資料
    
    實際 HW7 使用類似 SQuAD 格式的中文問答資料
    """
    samples = [
        {
            "question": "深度學習是什麼？",
            "context": "深度學習是機器學習的一個分支，使用多層神經網路來學習資料的表示。它在影像辨識、自然語言處理等領域取得了突破性進展。",
            "answer": "機器學習的一個分支",
            "answer_start": 4
        },
        {
            "question": "Transformer 是在哪一年提出的？",
            "context": "Transformer 架構是由 Google 在 2017 年的論文《Attention is All You Need》中提出的，它完全基於自注意力機制。",
            "answer": "2017 年",
            "answer_start": 18
        },
        {
            "question": "BERT 的全稱是什麼？",
            "context": "BERT 的全稱是 Bidirectional Encoder Representations from Transformers，它是一種預訓練語言模型。",
            "answer": "Bidirectional Encoder Representations from Transformers",
            "answer_start": 12
        },
    ]
    
    # 複製以達到所需數量
    while len(samples) < n_samples:
        samples = samples + samples
    
    return samples[:n_samples]

qa_data = generate_qa_data(100)
print(f"資料集大小: {len(qa_data)}")
print(f"\n範例:")
print(f"  問題: {qa_data[0]['question']}")
print(f"  上下文: {qa_data[0]['context'][:50]}...")
print(f"  答案: {qa_data[0]['answer']}")

In [ ]:
# ========== QA Dataset ==========

class QADataset(Dataset):
    """
    Extractive QA Dataset
    
    處理問答資料，產生模型輸入和標籤
    """
    
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Tokenize
        encoding = self.tokenizer(
            item['question'],
            item['context'],
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )
        
        # 找到答案的 token 位置
        # 這是簡化版本，實際需要更複雜的處理
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        token_type_ids = encoding['token_type_ids'].squeeze()
        
        # 簡化：假設答案在特定位置
        start_position = min(item['answer_start'], self.max_len - 1)
        end_position = min(start_position + len(item['answer']), self.max_len - 1)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'start_position': torch.tensor(start_position),
            'end_position': torch.tensor(end_position),
        }

if HAS_TRANSFORMERS:
    dataset = QADataset(qa_data, tokenizer, max_len=128)
    sample = dataset[0]
    print("Dataset 樣本:")
    for k, v in sample.items():
        print(f"  {k}: {v.shape if hasattr(v, 'shape') else v}")

In [ ]:
# ========== QA 模型 ==========

class BertQA(nn.Module):
    """
    BERT for Extractive QA
    
    在 BERT 輸出上加兩個線性層：
    - 一個預測起始位置
    - 一個預測結束位置
    """
    
    def __init__(self, model_name='bert-base-chinese'):
        super().__init__()
        
        if HAS_TRANSFORMERS:
            self.bert = BertModel.from_pretrained(model_name)
            hidden_size = self.bert.config.hidden_size
        else:
            # 模擬
            hidden_size = 768
            self.bert = None
        
        # 起始和結束位置的分類頭
        self.qa_outputs = nn.Linear(hidden_size, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        if self.bert is not None:
            outputs = self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            sequence_output = outputs.last_hidden_state
        else:
            # 模擬輸出
            batch_size, seq_len = input_ids.shape
            sequence_output = torch.randn(batch_size, seq_len, 768)
        
        # 預測起始和結束位置
        logits = self.qa_outputs(sequence_output)  # (batch, seq_len, 2)
        start_logits, end_logits = logits.split(1, dim=-1)
        
        start_logits = start_logits.squeeze(-1)  # (batch, seq_len)
        end_logits = end_logits.squeeze(-1)
        
        return start_logits, end_logits

if HAS_TRANSFORMERS:
    qa_model = BertQA()
    print(f"QA 模型參數量: {sum(p.numel() for p in qa_model.parameters()):,}")

In [ ]:
# ========== 訓練 QA 模型 ==========

def train_qa_model(model, train_loader, epochs=3, lr=2e-5):
    """
    訓練 QA 模型
    
    HW7 訓練技巧:
    - 較小的學習率 (2e-5)
    - 線性學習率 warmup
    - Gradient clipping
    """
    model = model.to(device)
    
    # 優化器（區分 BERT 和分類頭的學習率）
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() 
                      if not any(nd in n for nd in no_decay)],
            'weight_decay': 0.01
        },
        {
            'params': [p for n, p in model.named_parameters() 
                      if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)
    
    # 學習率調度
    total_steps = len(train_loader) * epochs
    warmup_steps = total_steps // 10
    
    # 損失函數
    criterion = nn.CrossEntropyLoss()
    
    history = {'loss': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            start_positions = batch['start_position'].to(device)
            end_positions = batch['end_position'].to(device)
            
            optimizer.zero_grad()
            
            start_logits, end_logits = model(
                input_ids, attention_mask, token_type_ids
            )
            
            # 計算損失
            start_loss = criterion(start_logits, start_positions)
            end_loss = criterion(end_logits, end_positions)
            loss = (start_loss + end_loss) / 2
            
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        history['loss'].append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return model, history

if HAS_TRANSFORMERS:
    train_loader = DataLoader(dataset, batch_size=4, shuffle=True)
    # 由於 BERT 較大，這裡只示範訓練流程
    print("訓練流程已定義。實際訓練需要足夠的 GPU 記憶體。")

In [ ]:
# ========== QA 推理 ==========

def answer_question(model, tokenizer, question, context, device='cpu'):
    """
    使用訓練好的模型回答問題
    """
    model.eval()
    
    # Tokenize
    encoding = tokenizer(
        question,
        context,
        max_length=512,
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    
    with torch.no_grad():
        start_logits, end_logits = model(
            input_ids, attention_mask, token_type_ids
        )
    
    # 找到最可能的起始和結束位置
    start_idx = start_logits.argmax().item()
    end_idx = end_logits.argmax().item()
    
    # 確保 end >= start
    if end_idx < start_idx:
        end_idx = start_idx
    
    # 解碼答案
    answer_ids = input_ids[0, start_idx:end_idx+1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    
    return answer, start_idx, end_idx

if HAS_TRANSFORMERS:
    print("QA 推理函數已定義")

---
## Part 4: BERT 微調技巧

In [ ]:
# ========== 凍結部分層 ==========

def freeze_bert_layers(model, num_layers_to_freeze=6):
    """
    凍結 BERT 的前幾層
    
    技巧: 凍結底層，只訓練高層和分類頭
    可以加速訓練並減少過擬合
    """
    # 凍結 embeddings
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False
    
    # 凍結前 N 層
    for i in range(num_layers_to_freeze):
        for param in model.bert.encoder.layer[i].parameters():
            param.requires_grad = False
    
    # 計算可訓練參數
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    
    print(f"可訓練參數: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")

if HAS_TRANSFORMERS:
    print("凍結前 6 層:")
    test_model = BertQA()
    freeze_bert_layers(test_model, num_layers_to_freeze=6)

In [ ]:
# ========== 分層學習率 ==========

def get_layerwise_lr_params(model, base_lr=2e-5, lr_decay=0.95):
    """
    為不同層設置不同的學習率
    
    底層學習率較小，高層學習率較大
    """
    params = []
    
    if not HAS_TRANSFORMERS or model.bert is None:
        return [{'params': model.parameters(), 'lr': base_lr}]
    
    # BERT 層（從高到低）
    num_layers = model.bert.config.num_hidden_layers
    
    for i in range(num_layers - 1, -1, -1):
        layer_lr = base_lr * (lr_decay ** (num_layers - 1 - i))
        params.append({
            'params': model.bert.encoder.layer[i].parameters(),
            'lr': layer_lr
        })
    
    # Embeddings 使用最小學習率
    params.append({
        'params': model.bert.embeddings.parameters(),
        'lr': base_lr * (lr_decay ** num_layers)
    })
    
    # 分類頭使用最大學習率
    params.append({
        'params': model.qa_outputs.parameters(),
        'lr': base_lr
    })
    
    return params

print("分層學習率配置完成")

---
## Part 5: HW7 實用技巧

### 處理長文本

In [ ]:
# ========== Sliding Window 處理長文本 ==========

def process_long_context(question, context, tokenizer, max_len=512, doc_stride=128):
    """
    使用滑動窗口處理超長上下文
    
    當文章太長時，將其分割成多個重疊的窗口
    """
    # 先 tokenize 問題
    question_tokens = tokenizer.tokenize(question)
    
    # 可用於上下文的最大長度
    # 需要扣除 [CLS], [SEP], [SEP] 和問題長度
    max_context_len = max_len - len(question_tokens) - 3
    
    # Tokenize 上下文
    context_tokens = tokenizer.tokenize(context)
    
    if len(context_tokens) <= max_context_len:
        # 不需要分割
        return [{'question': question, 'context': context, 'offset': 0}]
    
    # 滑動窗口分割
    windows = []
    start = 0
    
    while start < len(context_tokens):
        end = min(start + max_context_len, len(context_tokens))
        
        window_tokens = context_tokens[start:end]
        window_context = tokenizer.convert_tokens_to_string(window_tokens)
        
        windows.append({
            'question': question,
            'context': window_context,
            'offset': start
        })
        
        if end >= len(context_tokens):
            break
        
        start += doc_stride
    
    return windows

if HAS_TRANSFORMERS:
    # 測試
    long_context = "深度學習是機器學習的一個分支。" * 100
    windows = process_long_context(
        "什麼是深度學習？",
        long_context,
        tokenizer,
        max_len=128,
        doc_stride=64
    )
    print(f"原始上下文長度: {len(tokenizer.tokenize(long_context))} tokens")
    print(f"分割成 {len(windows)} 個窗口")

In [ ]:
# ========== 答案後處理 ==========

def post_process_predictions(start_logits, end_logits, n_best=5, max_answer_len=30):
    """
    後處理預測結果
    
    1. 找到 n_best 個最可能的 (start, end) 組合
    2. 過濾不合法的組合（end < start, 長度超限）
    3. 返回得分最高的答案
    """
    # 獲取 top k 的起始和結束位置
    start_indices = torch.topk(start_logits, n_best).indices.tolist()
    end_indices = torch.topk(end_logits, n_best).indices.tolist()
    
    candidates = []
    
    for start_idx in start_indices:
        for end_idx in end_indices:
            # 過濾條件
            if end_idx < start_idx:
                continue
            if end_idx - start_idx + 1 > max_answer_len:
                continue
            
            score = start_logits[start_idx] + end_logits[end_idx]
            candidates.append({
                'start': start_idx,
                'end': end_idx,
                'score': score.item()
            })
    
    # 按分數排序
    candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)
    
    return candidates[:n_best]

# 測試
test_start = torch.randn(128)
test_end = torch.randn(128)
candidates = post_process_predictions(test_start, test_end)
print(f"Top {len(candidates)} 候選答案:")
for i, c in enumerate(candidates[:3]):
    print(f"  {i+1}. start={c['start']}, end={c['end']}, score={c['score']:.2f}")

---
## 🎯 總結

### BERT 微調要點

| 技巧 | 說明 |
|------|------|
| **小學習率** | 2e-5 ~ 5e-5，避免破壞預訓練權重 |
| **Warmup** | 前 10% 步數線性增加學習率 |
| **Gradient Clipping** | 防止梯度爆炸 |
| **分層學習率** | 底層小，高層大 |
| **凍結層** | 凍結底層加速訓練 |

### Extractive QA 流程

```
訓練:
1. Tokenize 問題 + 上下文
2. 標記答案起始和結束位置
3. 用 CrossEntropy 訓練兩個分類頭

推理:
1. 長文本使用滑動窗口
2. 預測每個窗口的 (start, end)
3. 後處理選擇最佳答案
```

### 李宏毅 HW7 技巧

```
Strong Baseline:
1. 使用 chinese-bert-wwm 等中文預訓練模型
2. 正確處理滑動窗口
3. Learning rate warmup
4. Gradient accumulation（如果 batch size 太小）
5. 答案後處理（n-best, 長度限制）
```

### 下一步

前往 `robustness_security/explainable_ai.ipynb` 學習可解釋 AI！

---
## 🏋️ 練習

### 練習 1: 視覺化 BERT 的注意力權重

觀察 BERT 如何關注文本的不同部分是理解模型的重要方式。實作一個函數來提取和視覺化注意力。

In [ ]:
# ========== 練習 1: 視覺化 BERT 注意力權重 ==========

def visualize_bert_attention(model, tokenizer, text, layer_idx=-1, head_idx=0):
    """
    視覺化 BERT 特定層和頭的注意力權重
    
    Args:
        model: BERT 模型
        tokenizer: BERT tokenizer
        text: 輸入文本
        layer_idx: 要視覺化的層索引（-1 表示最後一層）
        head_idx: 要視覺化的注意力頭索引
    """
    model.eval()
    
    # Tokenize
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
    
    # 獲取注意力權重
    with torch.no_grad():
        outputs = model(**encoding, output_attentions=True)
    
    # attentions: tuple of (batch, num_heads, seq_len, seq_len) for each layer
    attention = outputs.attentions[layer_idx][0, head_idx].cpu().numpy()
    
    # 繪製熱力圖
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(attention, cmap='Blues')
    
    # 設置標籤
    ax.set_xticks(range(len(tokens)))
    ax.set_yticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=90, fontsize=8)
    ax.set_yticklabels(tokens, fontsize=8)
    
    ax.set_xlabel('Key Tokens')
    ax.set_ylabel('Query Tokens')
    ax.set_title(f'BERT Attention (Layer {layer_idx}, Head {head_idx})')
    
    plt.colorbar(im, ax=ax, label='Attention Weight')
    plt.tight_layout()
    plt.show()
    
    return attention

if HAS_TRANSFORMERS:
    # 測試視覺化
    print("練習 1: 視覺化 BERT 注意力權重")
    print("=" * 50)
    
    # 載入模型並啟用注意力輸出
    bert_model = BertModel.from_pretrained('bert-base-chinese', output_attentions=True)
    
    # 測試文本
    test_text = "深度學習是人工智慧的核心技術"
    
    # 視覺化最後一層的第一個注意力頭
    attention = visualize_bert_attention(bert_model, tokenizer, test_text, layer_idx=-1, head_idx=0)
    
    # 比較不同頭的注意力模式
    print("\n不同注意力頭可能學習不同的模式：")
    print("- 有些頭關注前一個/後一個詞（位置模式）")
    print("- 有些頭關注特定詞性（語法模式）")
    print("- 有些頭關注 [SEP] 或 [CLS]（全局模式）")
else:
    print("需要安裝 transformers 才能執行此練習")

### 練習 2: 比較不同 doc_stride 對長文本 QA 的影響

在處理長文本時，滑動窗口的步長（doc_stride）會影響覆蓋率和計算成本。實驗不同 stride 的效果。

In [ ]:
# ========== 練習 2: 比較不同 doc_stride 的影響 ==========

def experiment_doc_stride(question, context, tokenizer, max_len=128, strides=[32, 64, 96, 128]):
    """
    比較不同 doc_stride 對長文本處理的影響
    
    Args:
        question: 問題
        context: 長文本上下文
        tokenizer: BERT tokenizer
        max_len: 最大序列長度
        strides: 要測試的 stride 值列表
    """
    results = []
    
    context_tokens = tokenizer.tokenize(context)
    question_tokens = tokenizer.tokenize(question)
    max_context_len = max_len - len(question_tokens) - 3
    
    print(f"問題: {question}")
    print(f"上下文長度: {len(context_tokens)} tokens")
    print(f"每個窗口最大上下文長度: {max_context_len} tokens")
    print("=" * 60)
    
    for stride in strides:
        windows = process_long_context(question, context, tokenizer, max_len=max_len, doc_stride=stride)
        
        # 計算覆蓋統計
        num_windows = len(windows)
        
        # 計算重疊率
        if num_windows > 1:
            overlap = max_context_len - stride
            overlap_ratio = overlap / max_context_len
        else:
            overlap_ratio = 0.0
        
        # 計算計算成本（相對於單窗口）
        compute_cost = num_windows
        
        results.append({
            'stride': stride,
            'num_windows': num_windows,
            'overlap_ratio': overlap_ratio,
            'compute_cost': compute_cost
        })
        
        print(f"doc_stride={stride:3d}: {num_windows:3d} 窗口, "
              f"重疊率={overlap_ratio*100:5.1f}%, 計算成本={compute_cost}x")
    
    # 視覺化
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    stride_vals = [r['stride'] for r in results]
    
    axes[0].bar(range(len(stride_vals)), [r['num_windows'] for r in results], color='steelblue')
    axes[0].set_xticks(range(len(stride_vals)))
    axes[0].set_xticklabels(stride_vals)
    axes[0].set_xlabel('doc_stride')
    axes[0].set_ylabel('窗口數量')
    axes[0].set_title('窗口數量 vs doc_stride')
    
    axes[1].bar(range(len(stride_vals)), [r['overlap_ratio']*100 for r in results], color='coral')
    axes[1].set_xticks(range(len(stride_vals)))
    axes[1].set_xticklabels(stride_vals)
    axes[1].set_xlabel('doc_stride')
    axes[1].set_ylabel('重疊率 (%)')
    axes[1].set_title('重疊率 vs doc_stride')
    
    # 權衡分析：更多重疊 = 更高覆蓋但更多計算
    axes[2].scatter([r['num_windows'] for r in results], 
                   [r['overlap_ratio']*100 for r in results], 
                   c=stride_vals, cmap='viridis', s=100)
    for r in results:
        axes[2].annotate(f"s={r['stride']}", 
                        (r['num_windows'], r['overlap_ratio']*100 + 2))
    axes[2].set_xlabel('計算成本（窗口數量）')
    axes[2].set_ylabel('重疊率 (%)')
    axes[2].set_title('成本 vs 覆蓋率權衡')
    
    plt.tight_layout()
    plt.show()
    
    print("\n結論：")
    print("- 較小的 stride → 更多重疊 → 更好的覆蓋但更慢")
    print("- 較大的 stride → 更少重疊 → 可能漏掉跨窗口答案")
    print("- 建議：stride = max_len // 2 是常見的平衡選擇")
    
    return results

if HAS_TRANSFORMERS:
    print("練習 2: 比較不同 doc_stride 的影響")
    print("=" * 50)
    
    # 生成長文本
    long_context = """
    深度學習是機器學習的一個分支，它使用多層神經網路來學習資料的表示。
    這種方法在影像辨識、自然語言處理、語音識別等領域取得了突破性進展。
    BERT 是由 Google 在 2018 年提出的預訓練語言模型，全稱是 Bidirectional Encoder Representations from Transformers。
    它的核心創新是使用雙向 Transformer 編碼器和兩個預訓練任務：遮蓋語言模型和下一句預測。
    GPT 系列則是由 OpenAI 開發的自回歸語言模型，使用單向 Transformer 解碼器。
    Transformer 架構最初在 2017 年的論文《Attention is All You Need》中提出。
    自注意力機制是其核心組件，可以捕捉序列中任意位置之間的依賴關係。
    """ * 5  # 重複以創建長文本
    
    results = experiment_doc_stride(
        question="BERT 是什麼時候提出的？",
        context=long_context,
        tokenizer=tokenizer,
        max_len=128,
        strides=[32, 64, 96, 128]
    )
else:
    print("需要安裝 transformers 才能執行此練習")